<a href="https://colab.research.google.com/github/poojavjpy/Data-Analytics/blob/main/YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.8/874.8 kB 49.2 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import os
from ultralytics import YOLO




Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# To clear the **cache**

In [ ]:
!rm /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/train.cache

In [ ]:
!rm /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/val.cache

# Dataset Loading

In [ ]:
# Define the paths to your dataset and where to save the model
dataset_path = '/content/drive/MyDrive...'  # Path to your YOLOv5 formatted dataset
save_model_path = '/content/drive/MyDrive/..'  # Path to save the trained YOLOv8 model

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # 'yolov8n.pt' is the nano version of YOLOv8. You can use 'yolov8s.pt', 'yolov8m.pt', etc. for different sizes


100%|██████████| 6.25M/6.25M [00:00<00:00, 359MB/s]


# Training

In [ ]:
# Define the correct path to your dataset's data.yaml
dataset_yaml_path = '/content/drive/MyDrive/locust_detection/.../robo.dataset.yaml'  # Ensure this is the correct path

# Train the YOLOv8 model using the correct path
results = model.train(
    data=dataset_yaml_path,  # Use the correct path here
    epochs=60,
    batch=16,
    imgsz=640,
    save_period=10,
    project=save_model_path,
    name='yolov8_detection'
)


Ultralytics YOLOv8.2.102 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/locust_detection/yolov5/robo.dataset.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8_model/robo yolo v8, name=yolov8_detection24, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

100%|██████████| 755k/755k [00:00<00:00, 108MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/train... 1210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1210/1210 [04:16<00:00,  4.72it/s]


train: New cache created: /content/drive/MyDrive/locust_detection/dataset/roboflow/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      60/60      2.15G      1.223     0.6813      1.642         11        640: 100%|██████████| 76/76 [00:12<00:00,  6.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  5.26it/s]

                   all        341        368       0.91      0.823       0.87      0.461



60 epochs completed in 0.260 hours.
Optimizer stripped from /content/drive/MyDrive/yolov8_model/robo yolo v8/yolov8_detection24/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/yolov8_model/robo yolo v8/yolov8_detection24/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/yolov8_model/robo yolo v8/yolov8_detection24/weights/best.pt...
Ultralytics YOLOv8.2.102 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  3.69it/s]


                   all        341        368      0.925      0.805      0.876      0.463
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/yolov8_model/robo yolo v8/yolov8_detection24


# Checking Results

In [ ]:
import pandas as pd
from google.colab import drive
import os

# Mount Google Drive to access the results file
drive.mount('/content/drive')

def extract_metrics(file_path):
    # Read the CSV file from the provided file path
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()  # Clean up column names

        # Print the column names for inspection
        print("Columns in the CSV file:", df.columns.tolist())

        # Extract precision, recall, and mAP metrics from the CSV file
        precision = df['metrics/precision(B)'].values[-1]  # Get last epoch's precision
        recall = df['metrics/recall(B)'].values[-1]        # Get last epoch's recall
        map50 = df['metrics/mAP50(B)'].values[-1]          # Get last epoch's mAP50
        map50_95 = df['metrics/mAP50-95(B)'].values[-1]    # Get last epoch's mAP50-95

        return precision, recall, map50, map50_95

    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None  # Return None when the file is not found
    except KeyError as e:
        print(f"Error: Missing column {e} in the CSV file.")
        return None  # Return None if a column is missing
    except Exception as e:
        print(f"An error occurred: {e}")
        return None  # Return None for any other exception

# Provide the path to your YOLO results CSV file within Google Drive
file_path = '/content/drive/MyDrive/yolov8_model/robo yolo v8/yolov8_detection24/results.csv'  # Update with your actual file path

# Check if the file exists
if os.path.isfile(file_path):
    # Extract and display precision, recall, and mAP metrics
    metrics = extract_metrics(file_path)
    if metrics is None:
        print("Failed to extract metrics.")
    else:
        precision, recall, map50, map50_95 = metrics
        print(f"Precision: {precision * 100:.2f}%")
        print(f"Recall: {recall * 100:.2f}%")
        print(f"mAP@50: {map50 * 100:.2f}%")
        print(f"mAP@50-95: {map50_95 * 100:.2f}%")
else:
    print(f"Error: File not found at {file_path}. Please check the path.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columns in the CSV file: ['epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']
Precision: 90.97%
Recall: 82.34%
mAP@50: 87.01%
mAP@50-95: 46.12%


In [ ]:
from ultralytics import YOLO
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
weights_path = '/content/drive/MyDrive/../weights/last.pt'  # Path to the trained model weights
source_directory = '/content/drive/MyDrive/locust_detection/test_data/images'  # Directory containing images
output_directory = '/content/drive/MyDrive/yolov8_output/robo yolo v8'  # Directory to save output images

# Load the YOLOv8 model
model = YOLO(weights_path)

# Perform inference on all images in the source directory
results = model.predict(source=source_directory, conf=0.40, save=True, project=output_directory, name='detection_results')

# Move images if YOLOv8 saves them in subfolders
for root, dirs, files in os.walk(output_directory):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            src_path = os.path.join(root, file)
            dst_path = os.path.join(output_directory, file)
            shutil.move(src_path, dst_path)

# Function to resize images
def resize_image(image_path, size=(640, 480)):
    with Image.open(image_path) as img:
        img = img.resize(size, Image.Resampling.LANCZOS)  # Updated from ANTIALIAS to LANCZOS
        return img

# Display resized images
image_files = [f for f in os.listdir(output_directory) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
num_images = len(image_files)

# Set grid size
cols = 5
rows = (num_images // cols) + (num_images % cols > 0)  # Number of rows needed

plt.figure(figsize=(cols * 4, rows * 4))  # Adjust the figure size based on the grid

for i, image_file in enumerate(image_files):
    img_path = os.path.join(output_directory, image_file)
    img = resize_image(img_path)

    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Image {i + 1}")

plt.tight_layout()
plt.show()
